In [1]:
%matplotlib qt

In [3]:
import matplotlib as mpl
import sys
sys.path.append("/Users/yao/Smilei")
import happi
import numpy as np
import matplotlib.pyplot as plt

jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd
jet_vals = jetcmap(np.arange(9)) #extract those values as an array 
jet_vals[0] = [1.0, 1, 1.0, 1] #change the first value 
jet_vals[8] = [0.0, 0, 0.0, 1] #change the first value 
newcmap = mpl.colors.LinearSegmentedColormap.from_list("mine", jet_vals) 

from matplotlib import font_manager

font_dirs = ['/Users/yao/Documents/Calibri and Cambria Fonts/']
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

# set font
plt.rcParams['font.family'] = 'Calibri'

plt.rc('text', usetex=False)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('axes', labelsize=14)
plt.rc('legend', fontsize=11)

/var/folders/2t/97rc3fl92tg15k2l_4sk5hsh0000gn/T/ipykernel_53475/4105140122.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd


In [10]:
wkdir = [
    # '/Users/yao/Documents/Data/Apollon2024/stage2_t0',
    # '/Users/yao/Documents/Data/Apollon2024/stage2_t1',
    # '/Users/yao/Documents/Data/Apollon2024/stage2_t3_r0',
    # '/Users/yao/Documents/Data/Apollon2024/stage2_t4_r*',
    # '/Users/yao/Documents/Data/Apollon2024/stage2_t5_r*',
    # '/Users/yao/Documents/Data/Apollon2024/stage2_t6',
    # '/Users/yao/Documents/Data/Apollon2024/stage2_t7',
    # '/Users/yao/Documents/Data/Apollon2024/stage2_t8',
    '/Users/yao/Documents/Data/Apollon2024/apo24_t1',
    '/Users/yao/Desktop/apo24_sep17_t2',
    ]

In [11]:
S0 = happi.Open(wkdir[0], reference_angular_frequency_SI = 56375055300167.87)
S1 = happi.Open(wkdir[1], reference_angular_frequency_SI = 56375055300167.87)
# S2 = happi.Open(wkdir[2], reference_angular_frequency_SI = 56375055300167.87)
# S3 = happi.Open(wkdir[3], reference_angular_frequency_SI = 56375055300167.87)
# S4 = happi.Open(wkdir[4], reference_angular_frequency_SI = 56375055300167.87)

Loaded simulation '/Users/yao/Documents/Data/Apollon2024/apo24_t1'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics
Scanning for new particle diagnostics
Loaded simulation '/Users/yao/Desktop/apo24_sep17_t2'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics
Scanning for new particle diagnostics


In [12]:
# prepare constants, units

me = 9.1e-31
mp = 1836.*me
qe = 1.6e-19
ep = 8.9e-12  # epsilon_0
c  = 3.0e8
wr = S0.namelist.w_r
de = c / wr
Lx = S0.namelist.L_x.real * de * 1e3      # in mm
dx = S0.namelist.d_x * de * 1e3           # in mm

Te = S0.namelist.T_e * 511.e3             # in eV
ne = 1.0e18                              # in cm-3
ld = 7.43e2 * Te**0.5 * ne**(-0.5) * 10. # in mm
dt = S0.namelist.d_t

B0 = S0.namelist.B_z * (me * wr / qe)
wc = qe * B0 / me

# conversion constants for fields and momentum (when taken from TrackParticle)
kE = me*c*wr/qe  # V/m
kp = me*c        # kg m/s
kB = me*wr/qe    # T
conv_MeV_J = 10**6*qe

In [8]:
print(Lx)
print(dx*1e3*1e3)

7.999452906942997
21.28601016195237


In [ ]:
wr0 = np.sqrt(1e19*1e6*qe**2/(me*ep))
wr1 = 56375055300167.87
print('plasma frequency: %.5e rad/s'%(wr0))
print('plasma frequency: %.5e rad/s'%(wr1))

B_norm = (me * wr0 / qe)
print('B normalization: %.5e T'%(B_norm))

B0 = 40.0 # in T
Bz0 = B0 / B_norm
print('Bz0 = %.5f (in code unit)'%(Bz0))

tau1 = 1./wr1 * 1e12 * 2.5e3 # in ps
print('1/omega_r = %.2f ps'%(tau1))

tau0 = 1./wr0 * 1e12 * 8e3 # in ps
print('1/omega_r = %.2f ps'%(tau0))

plasma frequency: 1.77789e+14 rad/s
plasma frequency: 5.63751e+13 rad/s
B normalization: 1.01117e+03 T
Bz0 = 0.03956 (in code unit)
1/omega_r = 44.35 ps
1/omega_r = 45.00 ps


In [16]:
es0 = S0.ParticleBinning(2, units=['mm', 'fs', 'MeV','cm^-3'],
                #    sum={'x':'all'},
                  #  sum={'vx':'all'},
                   data_log=True,
                   color='red',
                   label='B=0',
                   xmax=20,
                   )#.slide()

es1 = S1.ParticleBinning(2, units=['mm', 'fs', 'MeV','cm^-3'],
                #    sum={'x':'all'},
                  #  sum={'vx':'all'},
                   data_log=True,
                   color='blue',
                   label='B=40 T',
                   xmax=20,
                   )#.slide()

happi.multiSlide(es0,es1)

/Users/yao/Smilei/happi/_Diagnostics/Diagnostic.py:857: RuntimeWarning: divide by zero encountered in log10
  return self._np.log10( self._dataLinAtTime(t) )


In [22]:
es0 = S0.ParticleBinning(3, units=['mm', 'fs', 'MeV','cm^-3'],
                #    sum={'x':'all'},
                  #  sum={'vx':'all'},
                  #  data_log=True,
                   cmap=newcmap,
                   xmin=3,xmax=5,
                #    color='red',
                #    label='B=0',
                #    xmax=20,
                   )#.slide()

es1 = S1.ParticleBinning(3, units=['mm', 'fs', 'MeV','cm^-3'],
                #    sum={'x':'all'},
                  #  sum={'vx':'all'},
                  #  data_log=True,
                   cmap=newcmap,
                   xmin=3,xmax=5,
                #    color='blue',
                #    label='B=40 T',
                #    xmax=20,
                   )#.slide()

happi.multiSlide(es0,es1,shape=[2,1])

In [6]:
Ex0 = S0.Field(1, 'Ex',units=['V/m','ps','mm'],
         cmap=newcmap,
        #  xmin=2,xmax=6,
         color='blue',
         vsym=True,
         label='1e18 cc',
        #  xmin=-0.1,xmax=11,
         )#.slide()

Ex1 = S1.Field(1, 'Ex',units=['V/m','ps','mm'],
         cmap=newcmap,
         vsym=True,
         color='red',
         linestyle='--',
         label='2e17 cc',
        #  xmin=2,xmax=6,
        #  xmin=-0.1,xmax=11,
         )#.slide()

Ex2 = S2.Field(1, 'Ex',units=['V/m','ps','mm'],
         cmap=newcmap,
         vsym=True,
         color='cyan',
         linestyle=':',
         label='5e18 cc',
        #  xmin=2,xmax=6,
        #  xmin=-0.1,xmax=11,
         )#.slide()

Ex3 = S3.Field(1, 'Ex',units=['V/m','ps','mm'],
         cmap=newcmap,
         vsym=True,
         color='magenta',
         linestyle='-.',
         label='1e19 cc',
        #  xmin=2,xmax=6,
        #  xmin=-0.1,xmax=11,
         )#.slide()

happi.multiSlide(Ex0, Ex1, Ex2, Ex3)

In [7]:
Bz4 = S4.Field(0, 'Bz',units=['V/m','ps','mm','T'],
         cmap=newcmap,
         vsym=True,
         color='magenta',
         linestyle='-.',
         label='40 T',
        #  xmin=2,xmax=6,
        #  xmin=-0.1,xmax=11,
         ).slide()

Field diagnostic #0: Bz


In [8]:
# ion phase space (all species)
# 0: all species
# 9: protons from the piston in stage 1
# 10 : protons from the ambient in stage 2

diag_ID = 10

ps0 = S0.ParticleBinning(diag_ID,
                   units=['mm','keV','ps','1e3 km/s','cm^-3'],
                   data_log=True,
                   cmap=newcmap,
                   vmin=-2,
                   vmax=4,
                   xmin=0,xmax=8,
                  #  timestep_indices=[0,9],
                   ymin=-150,
                   ymax=150,
                #    vmin=-4,
                #    vmax=4,
                  #  sum={"vx":"all"},
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                #   ).slide(figsize=(12, 6))
                  )#.slide(figsize=(6, 3))

ps1 = S1.ParticleBinning(diag_ID,
                   units=['mm','keV','ps','1e3 km/s','cm^-3'],
                   data_log=True,
                   cmap=newcmap,
                   vmin=-2,
                   vmax=4,
                   xmin=0,xmax=8,
                  #  timestep_indices=[0,9],
                   ymin=-150,
                   ymax=150,
                #    vmin=-4,
                #    vmax=4,
                  #  sum={"vx":"all"},
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                #   ).slide(figsize=(12, 6))
                  )#.slide(figsize=(6, 3))

ps2 = S2.ParticleBinning(diag_ID,
                   units=['mm','keV','ps','1e3 km/s','cm^-3'],
                   data_log=True,
                   cmap=newcmap,
                   vmin=-2,
                   vmax=4,
                   xmin=0,xmax=8,
                  #  timestep_indices=[0,9],
                   ymin=-150,
                   ymax=150,
                #    vmin=-4,
                #    vmax=4,
                  #  sum={"vx":"all"},
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                #   ).slide(figsize=(12, 6))
                  )#.slide(figsize=(6, 3))

ps3 = S3.ParticleBinning(diag_ID,
                   units=['mm','keV','ps','1e3 km/s','cm^-3'],
                   data_log=True,
                   cmap=newcmap,
                   vmin=-2,
                   vmax=4,
                   xmin=0,xmax=8,
                  #  timestep_indices=[0,9],
                   ymin=-150,
                   ymax=150,
                #    vmin=-4,
                #    vmax=4,
                  #  sum={"vx":"all"},
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                #   ).slide(figsize=(12, 6))
                  )#.slide(figsize=(6, 3))

ps4 = S4.ParticleBinning(diag_ID,
                   units=['mm','keV','ps','1e3 km/s','cm^-3'],
                   data_log=True,
                   cmap=newcmap,
                   vmin=-2,
                   vmax=4,
                   xmin=0,xmax=8,
                  #  timestep_indices=[0,9],
                   ymin=-150,
                   ymax=150,
                #    vmin=-4,
                #    vmax=4,
                  #  sum={"vx":"all"},
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                #   ).slide(figsize=(12, 6))
                  )#.slide(figsize=(6, 3))

happi.multiSlide(ps1, ps2, ps3, ps4, shape=(2,2), figsize=(12, 12),
                #  movie='/Users/yao/Desktop/ps_all.mp4',dpi=300,fps=2
                 )

/Users/yao/Smilei/happi/_Diagnostics/Diagnostic.py:857: RuntimeWarning: divide by zero encountered in log10
  return self._np.log10( self._dataLinAtTime(t) )


In [18]:
# electron phase space
S2.ParticleBinning(5,
                   units=['mm','keV','ps','km/s'],
                   data_log=True,
                   cmap=newcmap,
                   aspect='equal',
                #    ymin=-3e5,
                #    ymax=3e5,
                #    vmin=-4,
                #    vmax=4,
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                  ).slide(figsize=(12, 6))


#5 - Number density of species # 2,5,8
    vx from -0.208333 to 0.208333 in 500 steps 
    vy from -0.208333 to 0.208333 in 500 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size and by grid_length[0]



In [14]:
# ion energy spectrum
# 6: C; 7: H1; 8: H2+H3
# es_C = S0.ParticleBinning(6,
#                   # '#8/0.002',
#                   # '#7/0.02',
#                    units=['mm','MeV','ps','km/s'],
#                    data_log=True,
#                    cmap=newcmap,
#                   #  vmin=-4,
#                   #  vmax=4,
#                    sum={'x':'all'},
#                    label='piston C',
#                   #  xmin=0,xmax=25,
#                 #    vmin=0,vmax=1,
#                   # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
#                   )#.slide()

# es_p1 = S0.ParticleBinning(7,
#                   # '#8/0.002',
#                   # '#7/0.02',
#                    units=['mm','MeV','ps','km/s'],
#                    data_log=True,
#                    cmap=newcmap,
#                    label='piston H',
#                   #  vmin=-4,
#                   #  vmax=4,
#                    sum={'x':'all'},
#                   #  xmin=0,xmax=25,
#                 #    vmin=0,vmax=1,
#                   # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
#                   )#.slide()

diag_ID = 8

es_p0 = S0.ParticleBinning(diag_ID,
                  # '#8/0.002',
                  # '#7/0.02',
                   units=['mm','MeV','ps','km/s'],
                   data_log=True,
                   cmap=newcmap,
                    label='1e18 cc',
                    # timestep_indices=8,
                  #  vmin=-4,
                  #  vmax=4,
                   sum={'x':'all'},
                  #  xmin=0,xmax=25,
                #    vmin=0,vmax=1,
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                  )#.slide()

es_p1 = S1.ParticleBinning(diag_ID,
                  # '#8/0.002',
                  # '#7/0.02',
                   units=['mm','MeV','ps','km/s'],
                   data_log=True,
                   cmap=newcmap,
                  #  timestep_indices=8,
                    label='2e17 cc',
                  #  vmin=-4,
                  #  vmax=4,
                   sum={'x':'all'},
                  #  xmin=0,xmax=25,
                #    vmin=0,vmax=1,
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                  )#.slide()

es_p2 = S2.ParticleBinning(diag_ID,
                  # '#8/0.002',
                  # '#7/0.02',
                   units=['mm','MeV','ps','km/s'],
                   data_log=True,
                   cmap=newcmap,
                  #  timestep_indices=8,
                    label='5e18 cc',
                  #  vmin=-4,
                  #  vmax=4,
                   sum={'x':'all'},
                  #  xmin=0,xmax=25,
                #    vmin=0,vmax=1,
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                  )#.slide()

es_p3 = S3.ParticleBinning(diag_ID,
                  # '#8/0.002',
                  # '#7/0.02',
                   units=['mm','MeV','ps','km/s'],
                   data_log=True,
                   cmap=newcmap,
                  #  timestep_indices=8,
                    label='1e19 cc, 40 T',
                  #  vmin=-4,
                  #  vmax=4,
                   sum={'x':'all'},
                  #  xmin=0,xmax=25,
                #    vmin=0,vmax=1,
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                  )#.slide()

es_p4 = S4.ParticleBinning(diag_ID,
                  # '#8/0.002',
                  # '#7/0.02',
                   units=['mm','MeV','ps','km/s'],
                   data_log=True,
                   cmap=newcmap,
                  #  timestep_indices=8,
                    label='1e19 cc, 160 T',
                  #  vmin=-4,
                  #  vmax=4,
                   sum={'x':'all'},
                  #  xmin=0,xmax=25,
                #    vmin=0,vmax=1,
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                  )#.slide()

happi.multiSlide(es_p1, es_p0, es_p2, es_p3, es_p4)


/Users/yao/Smilei/happi/_Diagnostics/Diagnostic.py:857: RuntimeWarning: divide by zero encountered in log10
  return self._np.log10( self._dataLinAtTime(t) )


In [6]:
# ion densities (all species)
n0 = S0.ParticleBinning(0,
                   units=['mm','keV','ps','km/s','cm^-3'],
                   data_log=True,
                   cmap=newcmap,
                   label='1e18 cc',
                #    vmin=-4,
                #    vmax=4,
                   sum={'vx':'all'},
                  #  xmin=0,xmax=25,
                  #  vmin=0,vmax=1,
                  # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
                  ).slide()

# n1 = S1.ParticleBinning(0,
#                    units=['mm','keV','ps','km/s','cm^-3'],
#                    data_log=True,
#                    cmap=newcmap,
#                    label='2e17 cc',
#                 #    vmin=-4,
#                 #    vmax=4,
#                    sum={'vx':'all'},
#                   #  xmin=0,xmax=25,
#                   #  vmin=0,vmax=1,
#                   # ).animate(movie='/Users/yao/Desktop/ps.gif',dpi=100,saveAs='/Users/yao/Desktop/tmp/ps.png')
#                   )#.slide()

# happi.multiSlide(n0, n1)


#0 - Number density of species # 0,1,3,6
    x from 0 to 2048 in 500 steps 
    vx from -0.416667 to 0.416667 in 500 steps 

average for all vx
The value in each bin is the sum of the `deposited_quantity` divided by the bin size

